In [ ]:
import requests
from shapely.geometry import LineString
import pandas as pd
import json
import osmnx as ox
import pyrosm
import folium
import numpy as np

ox.__version__

useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 'traffic_signals']
ox.utils.config(useful_tags_path)

In [ ]:
ADDRESS = "Mayfair, London"
G = ox.graph_from_address(ADDRESS, network_type="drive", dist=300)

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
def get_full_traffic_data(edges):
    
    names = edges.name.value_counts().index
    
    for name in names:
        get_specific_traffic_data(edges, name)
        
    return edges

def get_specific_traffic_data(edges, name):
    # Create request URL
    BASE_URL = "https://api.tomtom.com/"
    API_KEY = "ThfURaOhGRaty7mx9AKALwrbyIZApHd8"
    #zoom = 22
    TRAFFIC_URL = "traffic/services/4/flowSegmentData/absolute/22/json?key="
    UNIT = "unit=kmph"
    ROAD_CLOSURE = "roadClosure=True"
    PARAMS = "&" + UNIT + "&" + ROAD_CLOSURE
    
    coordinates = list(edges.loc[edges['name'] == name].iloc[0].geometry.coords)[0]
    longitude = str(coordinates[0])
    latitude = str(coordinates[1])
    location = "point=" + latitude + ","+ longitude

    request_url = BASE_URL + TRAFFIC_URL + API_KEY + "&" + location + PARAMS
    # Get data
    response = requests.get(request_url)
    data = response.json()['flowSegmentData']
    traffic = pd.json_normalize(data)
    
    edges.loc[edges['name'] == name, 'road_type'] = traffic.frc[0]
    edges.loc[edges['name'] == name, 'current_speed_kmph'] = traffic.currentSpeed[0]
    edges.loc[edges['name'] == name, 'free_flow_speed_kmph'] = traffic.freeFlowSpeed[0]
    edges.loc[edges['name'] == name, 'traffic_confidence'] = traffic.confidence[0]
    edges.loc[edges['name'] == name, 'road_closure'] = traffic.roadClosure[0]

In [ ]:
#edges.loc[edges['name'] == 'Davies Street']
edges

In [ ]:
get_specific_traffic_data(edges, "Davies Street")

In [ ]:
#edges.loc[edges['name'] == 'Davies Street']
edges

In [ ]:
#print(edges.name.value_counts())

In [ ]:
#edges.loc[edges['name'].astype(str) == ['Hanover Square', 'Brook Street']]
# z = edges[edges['name'].apply(lambda x: isinstance(x, list))]
# z

In [ ]:
# names = edges.name.value_counts().index
# names